In [1]:
from fontMatching.fontGen import *
from fontMatching.model import *
from visualize import optimize2
from os import walk
import json
from ocr import interogate, add_rect 
import math
from bs4 import BeautifulSoup
from svgpathtools import svg2paths2
import webcolors

c:\research\svg-typography-rust\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
feature = get_text_feature("./pngs/Blush Modern Stylish New Post Instagram Post.svg.png", ["NEW", "POST", "swipe", "up"])
print(feature)
for k,v in feature.items():
    color_hex = webcolors.hex_to_rgb(v["color"])
    background_hex = webcolors.hex_to_rgb(v["backgroundColor"])
    v["color"] = (color_hex.red, color_hex.green, color_hex.blue)
    v["backgroundColor"] = (background_hex.red, background_hex.green, background_hex.blue)


feature

{'NEW': {'color': '#000000', 'fontSize': '30px', 'backgroundColor': '#F5F5DC'}, 'POST': {'color': '#000000', 'fontSize': '30px', 'backgroundColor': '#F5F5DC'}, 'swipe': {'color': '#A0522D', 'fontSize': '20px', 'backgroundColor': '#F5F5DC'}, 'up': {'color': '#A0522D', 'fontSize': '20px', 'backgroundColor': '#F5F5DC'}}


{'NEW': {'color': (0, 0, 0),
  'fontSize': '30px',
  'backgroundColor': (245, 245, 220)},
 'POST': {'color': (0, 0, 0),
  'fontSize': '30px',
  'backgroundColor': (245, 245, 220)},
 'swipe': {'color': (160, 82, 45),
  'fontSize': '20px',
  'backgroundColor': (245, 245, 220)},
 'up': {'color': (160, 82, 45),
  'fontSize': '20px',
  'backgroundColor': (245, 245, 220)}}

In [6]:
text = {
    "text": "NEW",
    "x": 4,
    "y": 0,
    "color": (0,255,0),
    "fontSize": 18,
}
images = create_text(text, (200, 20))
for i in images:
    display(i)

In [ ]:
for _,_,files in walk("./svgs"):
    for file in files:
        json_dict = add_rect(file)
        ocr_dict = interogate(f"./pngs/{file}.png")

        if (not ocr_dict): continue

        with open(f"./id_svg/{file}_withID.svg") as f:
            soup = BeautifulSoup(f, "xml")
            root = soup.find("svg")
            _,_, attr = svg2paths2(f"./id_svg/{file}_withID.svg")

        dim = attr["viewBox"].split(" ")

        svg_width = float(dim[2].replace("px",""))
        svg_height = float(dim[3].replace("px",""))

        m,n = len(json_dict), len(ocr_dict)

        json_map = {i : key for key, i in zip(list(json_dict.keys()), range(m))}
        ocr_map = {i : key for key, i in zip(list(ocr_dict.keys()), range(n))}

        bounds_json = list(json_dict.values())
        bounds_ocr = list(ocr_dict.values())
        bounds_ocr = [(left * svg_width, right * svg_width, top * svg_height, bottom * svg_height) for left,right,top,bottom in bounds_ocr]

        center_json = [((b[1] + b[0]) / 2, (b[2] + b[3]) / 2) for b in bounds_json]
        center_ocr = [((b[1] + b[0]) / 2, (b[2] +  b[3]) / 2) for b in bounds_ocr]

        d = [[ 0 for i in range(n) ] for j in range(m)]


        for i in range(m):
            x1,y1 = center_json[i][0], center_json[i][1]
            for j in range(n):
                x2,y2 = center_ocr[j][0], center_ocr[j][1]

                d[i][j] = math.sqrt((x2-x1) ** 2 + (y2-y1)**2)

        x = [ [0] * (n) for i in range(m) ]
        
        optimize2(bounds_ocr, center_json, x)


        # Match path assignments with OCR descriptions
        group = [[] for _ in range(n)]
        for i in range(m):
            if max(x[i]):
                group[x[i].index(1)].append(json_map[i])


        texts = []

        for text_index, e_list in enumerate(group):
            for element_id in e_list:
                e = soup.find(id=element_id)
                if e: e.decompose()
            
            text_content = ocr_map[text_index]

            x,y = center_ocr[text_index]
            replacement = soup.new_tag("text",x=x,y=y, color="green")
            replacement.insert(0, text_content)
            cur_object = {
                "x": x,
                "y": y,
                "text": text_content,
            }

            root.append(replacement)
            
        

        with open(f"./finalized_svg/{file}_final.svg", "wb") as wf:
            wf.write(soup.prettify("utf-8"))

1080 1080 ./pngs/Blush Modern Stylish New Post Instagram Post.svg.png
[['object_18', 'object_14', 'object_22'], ['object_4', 'object_26', 'object_1', 'object_30', 'object_34', 'object_38', 'object_2'], ['object_42', 'object_50', 'object_58', 'object_46', 'object_54'], ['object_62', 'object_66']]
1080 1080 ./pngs/chart.svg.png
[['object_468', 'object_472', 'object_464', 'object_480', 'object_476'], ['object_500', 'object_488', 'object_484', 'object_496', 'object_492'], ['object_504', 'object_508'], ['object_512', 'object_544', 'object_524', 'object_532', 'object_516', 'object_520', 'object_528', 'object_536', 'object_540'], ['object_548', 'object_556', 'object_552'], ['object_576', 'object_580', 'object_612', 'object_596', 'object_592', 'object_604', 'object_560', 'object_588', 'object_564', 'object_572', 'object_584', 'object_568', 'object_608', 'object_600'], ['object_628', 'object_620', 'object_648', 'object_632', 'object_636', 'object_624', 'object_644', 'object_652', 'object_616', 